# LSTM text generation

Example script to generate text from text files.

At least 20 epochs are required before the generated text starts sounding coherent.

It is recommended to run this script on GPU, as recurrent networks are quite computationally intensive.

If you try this script on new data, make sure your corpus has at least ~100k characters. ~1M is better.

In [1]:
from keras.callbacks import LambdaCallback
from keras.callbacks import EarlyStopping

from keras.models import Model, load_model

from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import concatenate

from keras.optimizers import Adam
from keras.utils import Sequence
from keras.utils.data_utils import get_file

import numpy as np
import random
import io

import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15337693887161334341
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17954772553750801969
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2842435971729058302
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5916393472
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3123601243359866979
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
class DataGenerator(Sequence):
    def __init__(self, text, char_indices, batch_size=128, maxlen=40, step=3):
        self.text = text
        self.char_indices = char_indices
        self.batch_size = batch_size
        self.maxlen = maxlen
        self.step = step

    def __len__(self):
        return ((len(self.text) - self.maxlen) // self.step) // self.batch_size

    def __getitem__(self, index):
        x = np.zeros((self.batch_size, self.maxlen, len(self.char_indices)), dtype=np.bool)
        y = np.zeros((self.batch_size, len(self.char_indices)), dtype=np.bool)

        for i in range(self.batch_size):
            idx = (i + index) * self.step

            for t, char in enumerate(self.text[idx: idx + self.maxlen]):
                x[i, t, self.char_indices[char]] = 1

            y[i, self.char_indices[self.text[idx + self.maxlen]]] = 1

        return x, y

In [3]:
path = get_file(
    'lovecraft.txt',
    origin='https://bashkirtsevich.pro/shared/lovecraft.txt'
)

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 13137488
total chars: 123


In [4]:
maxlen = 40  # cut the text in semi-redundant sequences of maxlen characters
training_generator = DataGenerator(text, char_indices, maxlen=maxlen)

## Build the model: a single LSTM

In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print(f'----- Generating text after Epoch: {epoch + 1}')

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print(f'----- diversity: {diversity}')

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print(f'----- Generating with seed: "{sentence}"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

        print(generated)
        
    model.save(f"model.h5")

In [6]:
# build/load the model: a single LSTM
model_path = None

if model_path:
    print('Load model...')
    model = load_model(model_path)
else:
    print('Build model...')

    num_chars = len(chars)

    vec = Input(shape=(maxlen, num_chars))
    l1 = LSTM(activation='tanh', return_sequences=True, units=128)(vec)
    l1_d = Dropout(0.2)(l1)

    input2 = concatenate([vec, l1_d])
    l2 = LSTM(activation='tanh', return_sequences=True, units=128)(input2)
    l2_d = Dropout(0.2)(l2)

    input3 = concatenate([vec, l2_d])
    l3 = LSTM(activation='tanh', return_sequences=True, units=128)(input3)
    l3_d = Dropout(0.2)(l3)

    input_d = concatenate([l1_d, l2_d, l3_d])
    
    l4 = LSTM(activation='tanh', return_sequences=False, units=128)(input_d)
    l4_d = Dropout(0.2)(l4)
    
    dense3 = Dense(units=num_chars)(l4_d)
    output_res = Activation('softmax')(dense3)
    
    model = Model(inputs=vec, outputs=output_res)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.), metrics=['accuracy'])
    

W0623 23:09:22.252649 140036356044608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0623 23:09:22.255193 140036356044608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0623 23:09:22.262649 140036356044608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Build model...


W0623 23:09:22.619403 140036356044608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0623 23:09:22.626896 140036356044608 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0623 23:09:23.500063 140036356044608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0623 23:09:23.509659 140036356044608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tenso

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 123)      0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 40, 128)      129024      input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 40, 128)      0           lstm_1[0][0]                     
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 40, 251)      0           input_1[0][0]                    
                                                                 dropout_1[0][0]                  
__________

In [ ]:
model.fit_generator(
    generator=training_generator,
    validation_data=training_generator,
    epochs=60,
    callbacks=[
        LambdaCallback(on_epoch_end=on_epoch_end),
        EarlyStopping(monitor="loss", min_delta=0.001, patience=3, mode="min")
    ],
)

W0623 23:09:23.867537 140036356044608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/60
 7218/34212 [=====>........................] - ETA: 2:27:46 - loss: 2.3440 - acc: 0.3064